<a href="https://colab.research.google.com/github/cabroderick/ML-AM-MQP/blob/main/Model_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!python --version

# uninstall improper package versions
!pip uninstall keras -y
!pip uninstall keras-nightly -y
!pip uninstall keras-Preprocessing -y
!pip uninstall keras-vis -y
!pip uninstall tensorflow -y
!pip uninstall h5py -y

# reinstall with proper versions
!pip install tensorflow==1.13.1
!pip install keras==2.0.8
!pip install h5py==2.10.0

# import mask rcnn and set up
%cd
!git clone https://github.com/matterport/Mask_RCNN.git
%cd Mask_RCNN/
!python setup.py install
!pip show mask-rcnn

from google.colab import drive
drive.mount('/content/drive')

In [22]:
# imports
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from mrcnn.config import Config
from mrcnn.model import MaskRCNN
from mrcnn.model import load_image_gt
from mrcnn import utils
from sklearn import metrics
import mrcnn.model as modellib
from mrcnn import visualize
from mrcnn.model import log
from matplotlib import pyplot
from matplotlib.patches import Rectangle
import cv2
import os
import json
import numpy as np
from mrcnn.utils import compute_recall

#%%

# specify paths for prediction
WEIGHTS_PATH = '/content/drive/MyDrive/MQP/custom_mrcnn.h5'
TEST_IMGS = []
CLASS_NAMES = ['gas entrapment porosity', 'lack of fusion porosity', 'keyhole porosity']

In [5]:

# configure inference model
class InferenceConfig(Config):
    NAME = 'inference'
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 4

cfg = InferenceConfig()

model = MaskRCNN(mode='inference',
                 config=cfg, model_dir='./')

model.load_weights(filepath=WEIGHTS_PATH, by_name=True)


In [64]:

class TestDataset(utils.Dataset):

    # define constants
    BASE_IMAGES_DIR = '/content/drive/MyDrive/MQP/Data/Eval/Images/' # directory where all images can be found
    BASE_ANNOTATIONS_DIR = '/content/drive/MyDrive/MQP/Data/Eval/Labels/' # directory where all images labels can be found
    IMAGES_DIRS = ['G0/', 'G9/', 'H0/', 'H4/', 'H5/', 'H6/', 'H8/', 'H9/', 'J0/', 'J3/', 'J4/', 'J7/',
                 'J8/', 'K0/', 'Q0/', 'Q3/', 'Q5/', 'Q9/', 'R2/', 'R6/', 'R7/'] # list of directories where images are contained
    ANNOTATIONS_DIRS = ['Labeled G0/', 'Labeled G9/', 'Labeled H0/', 'Labeled H4/', 'Labeled H5/', 'Labeled H6/',
                      'Labeled H8/', 'Labeled H9/', 'Labeled J0/', 'Labeled J3/', 'Labeled J4/',
                      'Labeled J7/', 'Labeled J8/', 'Labeled K0/', 'Labeled Q0/', 'Labeled Q3/', 'Labeled Q5/',
                      'Labeled Q9/', 'Labeled R2/', 'Labeled R6/', 'Labeled R7/'] # corresponding list of directories where annotations are contained

    CLASSES = ['lack of fusion porosity', 'keyhole porosity', 'other'] # all annotation classes

    def load_dataset(self):
        image_paths = []
        annotation_paths = []
        image_ids = []

        for i in range(len(self.IMAGES_DIRS)):
            image_paths.append([])
            annotation_paths.append([])
            image_ids.append([])
            i_dir = self.BASE_IMAGES_DIR + self.IMAGES_DIRS[i]
            a_dir = self.BASE_ANNOTATIONS_DIR + self.ANNOTATIONS_DIRS[i]
            for file in os.listdir(i_dir):
                i_id = file[:-4]
                image_ids[i].append(i_id)
                image_paths[i].append(i_dir+i_id+'.tif')
                if "20X_YZ" not in i_id:
                  annotation_paths[i].append(a_dir+i_id+'_20X_YZ.json')
                else:
                  annotation_paths[i].append(a_dir+i_id+'.json')

        if (len(image_paths) != len(annotation_paths)): # raise exception if mismatch betwaeen number of images and annotations
            raise(ValueError('Number of images and annotations must be equal'))
        # configure dataset
        for i in range(len(self.CLASSES)):
            self.add_class('dataset', i+1, self.CLASSES[i]) # add classes to model

        # add images and annotations to dataset, ensuring an even distribution
        for i in range(len(image_paths)):
            images = len(image_paths[i])
            for j in range(images):
              image_id = image_ids[i][j]
              image_path = image_paths[i][j]
              annotation_path = annotation_paths[i][j]

              mask, class_ids = self.extract_mask(image_path, annotation_path)
              print(image_path)

              self.add_image('dataset',
                                    image_id=image_id,
                                    path=image_path,
                                    mask=mask,
                                    class_ids=class_ids)
    '''
    Extracts a mask from an image
    image_id: The image id to extract the mask from
    Returns a mask and a corresponding list of class ids
    '''
    def load_mask(self, image_id):

        info = self.image_info[image_id] # extract image info from data added earlier
        mask = info['mask']
        class_ids = info['class_ids']

        return mask, class_ids

    def extract_mask(self, image_path, annotation_path):
      if not os.path.exists(annotation_path): # if the annotation path is not found, it is named differently than its source image
          annotation_path = annotation_path[:-5] + '_20X_YZ.json'
      if not os.path.exists(annotation_path): # if the annotation path is not found, it is named differently than its source image
          annotation_path = annotation_path.replace('_20X_YZ', "")

      print(image_path, annotation_path)

      f_ann = open(annotation_path,)
      annotation_json = json.load(f_ann)

      if not annotation_json['shapes']: # if there are no annotations to be extracted
          return [], [] # empty list return values will be ignored and thus image is ignored

      class_ids = []
      image = cv2.imread(image_path)
      height = image.shape[0]
      width = image.shape[1]

      annotation_list = []
      [annotation_list.append(shape) for shape in annotation_json['shapes'] if shape['shape_type'] =='rectangle'
      and self.normalize_classname(shape['label']) != 'gas entrapment porosity'] # get annotations in a list
      mask = np.zeros([height, width, len(annotation_list)], dtype='uint8') # initialize array of masks for each bounding box

      for i in range(len(annotation_list)):
        a = annotation_list[i]

        # extract row and col data and crop image to annotation size
        col_min, col_max = int(min(a['points'][0][0], a['points'][1][0])), int(max(a['points'][0][0], a['points'][1][0]))
        row_min, row_max = int(min(a['points'][0][1], a['points'][1][1])), int(max(a['points'][0][1], a['points'][1][1]))
        col_min, col_max, row_min, row_max = self.normalize_dimensions(col_min, col_max, row_min, row_max)
        cropped_img = image[row_min:row_max, col_min:col_max]  # crop image to size of bounding box
        cropped_img_gray = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2GRAY)
        edged = cv2.Canny(cropped_img_gray, 30, 200)

        # apply contour to image and fill
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
        dilated = cv2.dilate(edged, kernel)
        contours, hierarchy = cv2.findContours(dilated.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        polygon = np.zeros(cropped_img.shape)
        color = [255, 255, 255]
        cv2.fillPoly(polygon, contours, color)

        # normalize polygon to all boolean values and insert into mask
        polygon_bool = np.alltrue(polygon == color, axis=2)
        mask[row_min:row_max, col_min:col_max, i] = polygon_bool

        # draw contour and mask
        # cv2.drawContours(edged, contours, -1, (0, 255, 0), 1)
        # imS = cv2.resize(edged, (512, 512))
        # cv2.imshow('Contours', imS)
        # cv2.waitKey(0)
        # cv2.imshow('Polygon', cv2.resize(polygon, (512, 512)))
        # cv2.waitKey(0)

        # extract class id and append to list
        class_label = self.normalize_classname(a['label'])
        class_id = self.CLASSES.index(class_label)
        class_ids.append(class_id)

      return mask.astype(np.bool), np.array(class_ids, dtype=np.int32)

    def normalize_classname(self, class_name): # normalize the class name to one used by the model
      class_name = class_name.lower() # remove capitalization
      class_name = class_name.strip() # remove leading and trailing whitespace
      classes_dict = { # dictionary containing all class names used in labels and their appropriate model class name
        'gas entrapment porosity': 'gas entrapment porosity',
        'keyhole porosity': 'keyhole porosity',
        'lack of fusion porosity': 'lack of fusion porosity',
        'fusion porosity': 'lack of fusion porosity',
        'gas porosity': 'gas entrapment porosity',
        'lack-of-fusion': 'lack of fusion porosity',
        'keyhole': 'keyhole porosity',
        'other': 'other',
        'lack of fusion': 'lack of fusion porosity'
      }
      return classes_dict.get(class_name)

    '''
    Ensures extracted row and column coords are not out of bounds
    '''
    def normalize_dimensions(self, col_min, col_max, row_min, row_max):
      return max(col_min, 0), col_max, max(row_min, 0), row_max

In [67]:

def compute_ar(pred_boxes, gt_boxes, list_iou_thresholds):
    AR = []
    for iou_threshold in list_iou_thresholds:
        try:
            recall, _ = compute_recall(pred_boxes, gt_boxes, iou=iou_threshold)
            AR.append(recall)
        except:
            AR.append(0.0)
            pass
    AUC = 2 * (metrics.auc(list_iou_thresholds, AR))
    return AUC

    #Load dataset
dataset_val = TestDataset()
dataset_val.load_dataset()
dataset_val.prepare()

img_ids = dataset_val.image_ids

#%%

from mrcnn.utils import compute_ap
from mrcnn.model import mold_image

# make prediction & visualize
APs = []
ARs = []

for img_id in img_ids:
    print(img_id)
    print(APs)

    image, image_meta, gt_class_id, gt_bbox, gt_mask = load_image_gt(dataset_val, cfg, img_id, use_mini_mask=False)

    scaled_image = mold_image(image, cfg)

    sample = np.expand_dims(scaled_image, 0)

    yhat = model.detect(sample, verbose=0)

    r = yhat[0]

    AP, precisions, recalls, overlaps = compute_ap(gt_bbox, gt_class_id, gt_mask, r["rois"], r["class_ids"], r["scores"], r['masks'], iou_threshold=0.5)
    APs.append(AP)

    list_iou_thresholds = np.arange(0.5, 1.01, 0.1)
    AR = compute_ar(r['rois'], gt_bbox, list_iou_thresholds)
    ARs.append(AR)


mAP = np.mean(APs)
mAR = np.mean(ARs)

f1_score = 2 * ((mAP * mAR) / (mAP + mAR))

print("mAP: " + mAP)
print("mAR: " + mAR)
print("F1: " + f1_score)


/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_12.tif /content/drive/MyDrive/MQP/Data/Eval/Labels/Labeled G0/A1G0COL_12_20X_YZ.json
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_12.tif
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_315.tif /content/drive/MyDrive/MQP/Data/Eval/Labels/Labeled G0/A1G0COL_315_20X_YZ.json
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_315.tif
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_44.tif /content/drive/MyDrive/MQP/Data/Eval/Labels/Labeled G0/A1G0COL_44_20X_YZ.json
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_44.tif
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_514.tif /content/drive/MyDrive/MQP/Data/Eval/Labels/Labeled G0/A1G0COL_514_20X_YZ.json
/content/drive/MyDrive/MQP/Data/Eval/Images/G0/A1G0COL_514.tif
/content/drive/MyDrive/MQP/Data/Eval/Images/G9/A1G9COL_43.tif /content/drive/MyDrive/MQP/Data/Eval/Labels/Labeled G9/A1G9COL_43_20X_YZ.json
/content/drive/MyDrive/MQP/Data/Eval/Images/G9

/root/Mask_RCNN/mrcnn/utils.py:734: RuntimeWarning: invalid value encountered in true_divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


5
[0.0, 0.0, 0.0, 0.0, nan]


/root/Mask_RCNN/mrcnn/utils.py:734: RuntimeWarning: invalid value encountered in true_divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


6
[0.0, 0.0, 0.0, 0.0, nan, nan]


/root/Mask_RCNN/mrcnn/utils.py:734: RuntimeWarning: invalid value encountered in true_divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


7
[0.0, 0.0, 0.0, 0.0, nan, nan, nan]


/root/Mask_RCNN/mrcnn/utils.py:734: RuntimeWarning: invalid value encountered in true_divide
  recalls = np.cumsum(pred_match > -1).astype(np.float32) / len(gt_match)


8
[0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan]
9
[0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0]
10
[0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.10000000149011612]
11
[0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.10000000149011612, 0.08333333333333333]
12
[0.0, 0.0, 0.0, 0.0, nan, nan, nan, nan, 0.0, 0.10000000149011612, 0.08333333333333333, 0.20833333333333331]


RuntimeError: ignored